This notebbok is responsible for downloading the selected datasets, allowing us to improve our knowledge of them in terms of the distribution of sequence sizes, as well as the presence of possible duplicate sequences. 

# imports

In [0]:
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
import requests

# Download the data

In [0]:
# url = "https://github.com/chun-he-316/DeepAlgPro/blob/main/data/all.test.fasta?raw=true"
# response = requests.get(url)

# if response.status_code == 200:
#     with open("test_deepalgpro.fasta", "wb") as file:
#         file.write(response.content)
#     print("Download conclued with sucess.")
# else:
#      print(f"Error status: {response.status_code}")

In [0]:
# url = "https://github.com/chun-he-316/DeepAlgPro/blob/main/data/all.train.fasta?raw=true"
# response = requests.get(url)

# if response.status_code == 200:
#     with open("train_deepalgpro.fasta", "wb") as file:
#         file.write(response.content)
#     print("Download conclued with sucess.")
# else:
#      print(f"Error status: {response.status_code}")

In [0]:
# sequences = []
# labels = []

# with open('test_deepalgpro.fasta', 'r') as file:
#     lines = file.readlines()
    
#     for i in range(len(lines)):
#         label_line = lines[i].strip()
        
#         if label_line.startswith(">"):
#             if label_line.startswith(">allergen"):
#                 label = 1  # Allergenic
#             elif label_line.startswith(">non-allergen"):
#                 label = 0  # Non-allergenic
#             else:
#                 continue  

#             try:
#                 sequence_line = lines[i + 1].strip()
#                 sequences.append(sequence_line)
#                 labels.append(label)
#             except IndexError:
#                 print(f"Error to process lines {i} and {i+1}. Verify the file format.")
#                 continue

# # Create the DataFrame
# df_test = pd.DataFrame({'sequence': sequences, 'label': labels})

In [0]:
# sequences = []
# labels = []

# with open('train_deepalgpro.fasta', 'r') as file:
#     lines = file.readlines()
    
#     for i in range(len(lines)):
#         label_line = lines[i].strip()
        
#         if label_line.startswith(">"):
#             if label_line.startswith(">allergen"):
#                 label = 1  # Allergenic
#             elif label_line.startswith(">non-allergen"):
#                 label = 0  # Non-allergenic
#             else:
#                 continue  

#             try:
#                 sequence_line = lines[i + 1].strip()
#                 sequences.append(sequence_line)
#                 labels.append(label)
#             except IndexError:
#                 print(f"Error to process lines {i} and {i+1}. Verify the file format.")
#                 continue

# # Create the DataFrame
# df_train = pd.DataFrame({'sequence': sequences, 'label': labels})

#conect data

In [0]:
# df_allergenic = pd.concat([df_train, df_test], ignore_index=True)

# Save data

In [0]:
# x_test_deepalgpro=df_test.drop(['label'], axis=1)
# y_test_deepalgpro=df_test.loc[:,["label"]]

# x_test_deepalgpro.to_csv("x_test_deepalgpro.csv",index=False)
# y_test_deepalgpro.to_csv("y_test_deepalgpro.csv",index=False)

# x_train_deepalgpro=df_train.drop(['label'], axis=1)
# y_train_deepalgpro=df_train.loc[:,["label"]]
# x_train_deepalgpro.to_csv("x_train_deepalgpro.csv",index=False)
# y_train_deepalgpro.to_csv("y_train_deepalgpro.csv",index=False)

# Load data

__You can start from here__

In [0]:
x_train_deepalgpro_esm = pd.read_csv( 'x_train_deepalgpro.csv')
x_test_deepalgpro_esm = pd.read_csv('x_test_deepalgpro.csv')
y_train_deepalgpro_esm = pd.read_csv('y_train_deepalgpro.csv')
y_test_deepalgpro_esm = pd.read_csv('y_test_deepalgpro.csv')

# Shape

In [0]:
#different shapes
print('train_x_esm', x_train_deepalgpro_esm.shape)
print('test_x_esm', x_test_deepalgpro_esm.shape)
print('train_y_esm', y_train_deepalgpro_esm.shape)
print('test_y_esm', y_test_deepalgpro_esm.shape)


# Histogram

In [0]:

dataset_alllergenic = pd.concat([x_train_deepalgpro_esm, x_test_deepalgpro_esm])

dataset_alllergenic['sequence_length'] = dataset_alllergenic['sequence'].apply(len)

bins = range(0, dataset_alllergenic['sequence_length'].max() + 100, 100)  # Intervals of 100

plt.figure(figsize=(10,6))
plt.hist(dataset_alllergenic['sequence_length'], bins=bins, edgecolor='black')
plt.title('Overview of sequence length distribution (Interval Size: 100 amino-acids)',fontsize=16)
plt.xlabel('Length of sequences',fontsize=16)
plt.ylabel('Number of sequences',fontsize=16)
plt.savefig('allergenic_histogram.png', dpi=300)


plt.show()


# Data distribution

In [0]:
bins = range(0, dataset_alllergenic['sequence_length'].max() + 50, 50)

# Criar o gráfico com histograma e curva KDE
plt.figure(figsize=(10,6))

# Adicionar histograma com intervalo de 50
sns.histplot(dataset_alllergenic['sequence_length'], bins=bins, kde=False, color='skyblue', edgecolor='black', stat='density')

# Adicionar a curva KDE com ajuste de largura da banda
sns.kdeplot(dataset_alllergenic['sequence_length'], color='red', linewidth=1, bw_adjust=0.2)

# Adicionar título e rótulos
plt.title('Distribution of Sequence Lengths with KDE Curve')
plt.xlabel('Length of Sequences')
plt.ylabel('Density')

# Exibir o gráfico
plt.show()


# Verify the presence of duplicate sequences

In [0]:
duplicates = dataset_alllergenic['sequence'].duplicated()

num_duplicates = duplicates.sum()
print(f"Number of duplicate sequences: {num_duplicates}")

if num_duplicates > 0:
    duplicate_sequences = dataset_alllergenic[dataset_alllergenic['sequence'].duplicated(keep=False)]
    print("Duplicate sequences:")
    print(duplicate_sequences)
else:
    print("No duplicate sequences found.")

# Label values distribution

In [0]:
y_allergenic = pd.concat([y_train_deepalgpro_esm, y_test_deepalgpro_esm])

# Verificar a distribuição dos valores na coluna 'label'
label_counts = y_allergenic['label'].value_counts()

# Criar o gráfico circular
plt.figure(figsize=(8, 8))
wedges, texts, autotexts = plt.pie(label_counts, labels=label_counts.index, autopct='%1.1f%%', colors=plt.cm.Paired(range(len(label_counts))))

# Add the legend
plt.legend(wedges, label_counts.index, title="Labels", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1),fontsize=14)

# Set the title in English
plt.title('Class Distribution (Label 0 vs Label 1)',fontsize=16)
plt.savefig('allergenic_labels.png', dpi=300)

plt.show()

In [0]:
df_allergenic['Length'] = df_allergenic['sequence'].apply(len)

# Split the DataFrame by label
label_0_lengths = df_allergenic[df_allergenic['label'] == 0]['Length']
label_1_lengths = df_allergenic[df_allergenic['label'] == 1]['Length']

# Create histograms
plt.hist(label_0_lengths, alpha=0.5, label='Label 0', bins=10)
plt.hist(label_1_lengths, alpha=0.5, label='Label 1', bins=10)

plt.xlabel('Sequence length', fontsize=15)  # Change 14 to the desired font size
plt.ylabel('Frequency', fontsize=15)  
plt.title('Length distribution of protein sequences by label',fontsize=15)
plt.legend(fontsize=15)
plt.savefig('allergenic_histogram_labels.png', dpi=300)

plt.show()


In [0]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

# Exemplo de modelos
models = ['DeepAlgPro', 'AlgPred 2.0', 'AllerCatPro 2.0', 'AllerTOP V2', 'ProAllD', 'Propythia', 'OmniA']

# Exemplo de métricas (substitua pelos seus valores reais)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
values = {
    'Accuracy': [91.62, 90.28, 93.80, 89.79, 85.99, 93.90, 94.1],
    'Precision': [92.76, 85.14, 93.31, 91.24, 86.76, 93.94, 94.00],
    'Recall': [90.28, 97.61, 94.36, 88.03, 84.93, 93.94, 94.2],
    'F1 Score': [91.50, 90.90, 93.80, 89.60, 85.8, 93.94, 94.10],
}

# Usar uma paleta de cores mais agradável (tab10)
cmap = cm.get_cmap('Set1', len(models))
colors = cmap.colors

# Tamanho da figura
plt.figure(figsize=(12, 6))

# Número de barras por grupo
bar_width = 0.1  # Diminuir largura para dar mais espaço

# Espaço adicional entre grupos de métricas
spacing = 0.15

# Posições das barras no eixo x, considerando o espaçamento entre métricas
index = np.arange(len(metrics)) * (len(models) * bar_width + spacing)

# Criar barras para cada modelo
for i, model in enumerate(models):
    plt.bar(index + i * bar_width, [values[metric][i] for metric in metrics], 
            bar_width, label=model, color=colors[i])

# Configurações do gráfico
plt.xlabel('Metrics',fontsize=14)
plt.ylabel('Scores (%)',fontsize=14)
plt.title('Performance of different approaches on allergenic dataset', fontsize=14)

# Definir os ticks do eixo y de 5 em 5
plt.yticks(np.arange(0, 101, 5),fontsize=14)

# Configuração do eixo x
plt.xticks(index + bar_width * 3, metrics,fontsize=14)

# Exibir legenda fora do gráfico
plt.legend(title='Models', bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=13)

# Ajustar layout e mostrar gráfico
plt.tight_layout()
plt.savefig('allergenic_global.png', dpi=300)

plt.show()
